# Homework 8 Question 2: Paint production.
As part of its weekly production, a paint company produces five batches
of paints, always the same, for some big clients who have a stable demand. Every paint batch is
produced in a single production process, all in the same blender that needs to be cleaned between each
batch. The durations of blending paint batches 1 to 5 are 40, 35, 45, 32 and 50 minutes respectively.
The cleaning times depend of the colors and the paint types. For example, a long cleaning period is
required if an oil-based paint is produced after a water-based paint, or to produce white paint after a
dark color. The times are given in minutes in the following matrix $A$ where $A_{ij}$ denotes the cleaning
time after batch $i$ if it is followed by batch $j$. 

$$
A =
\begin{bmatrix}
   0& 11& 7& 13& 11 \\
    5& 0& 13& 15& 15 \\
    13& 15& 0& 23& 11 \\
9& 13&  5 & 0& 3 \\
3& 7& 7& 7& 0 \\
\end{bmatrix}
$$

Since the company has other activities, it wishes to deal with this weekly production in the shortest
possible time (blending and cleaning). What is the corresponding order of paint batches? The order
will be applied every week, so the cleaning time between the last batch of one week and the first of the
following week needs to be accounted for in the total duration of cleaning.

## Problem Data 

In [4]:
using NamedArrays
batches = [:one, :two, :three, :four, :five]
blendTimes = Dict(zip(batches,[40, 35, 45, 32 ,50]))
cleaningTimes = [ 0 11  7 13 11 ;
                  5  0 13 15 15 ;
                 13 15  0 23 11 ;
                  9 13  5  0  3 ;
                  3  7  7  7  0 ]
c = NamedArray(cleaningTimes,(batches,batches))
N = size(cleaningTimes,1);

## Problem Model 

In [10]:
using JuMP, Cbc

m = Model(solver = CbcSolver())
@variable(m, x[batches,batches], Bin)                                      # must formulate as IP this time
@constraint(m, c1[j in batches], sum( x[i,j] for i in batches ) == 1)      # one out-edge
@constraint(m, c2[i in batches], sum( x[i,j] for j in batches ) == 1)      # one in-edge
@constraint(m, c3[i in batches], x[i,i] == 0 )                            # no self-loops
@objective(m, Min, sum( x[i,j]*(c[i,j] + blendTimes[j]) for i in batches, j in batches ))   # minimize total cost
                                    
# MTZ variables and constraints
@variable(m, u[batches])
@constraint(m, c4[i in batches, j in batches[2:end]], u[i] - u[j] + N*x[i,j] <= N-1 );

In [11]:
status = solve(m)
println("Status: ", status)
xx = getvalue(x)
subtours = getAllSubtours(xx)
println("Total length of cleaning and blending: ", getobjectivevalue(m))
println(subtours)
# pretty print the solution
sol = NamedArray(zeros(Int,N,N),(batches,batches))
for i in batches
    for j in batches
        sol[i,j] = Int(xx[i,j])
    end
end
println(sol)

Status: Optimal
Total length of cleaning and blending: 243.0
Any[Symbol[:one,:four,:three,:five,:two,:one]]
5×5 Named Array{Int64,2}
A ╲ B │   one    two  three   four   five
──────┼──────────────────────────────────
one   │     0      0      0      1      0
two   │     1      0      0      0      0
three │     0      0      0      0      1
four  │     0      0      1      0      0
five  │     0      1      0      0      0


In [1]:
# HELPER FUNCTION: returns the cycle containing the city START.
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in batches
            if x[j,k] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x)
    nodesRemaining = batches
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end;